In [3]:
import torch
import pandas
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import json, itertools
torch.manual_seed(0)
tqdm.get_lock().locks = []

Just as a warm-up exercise let's confirm that quadratic-sized recurrent neural network is capable of reverting a simple sequence.
Just for the sake of simplicity the sequence will be one-hot encoded and put through a network with 2 recurrent layers of $seq\_length^2$ neurons and an output linear layer providing a final output with the next sequence item. 

TODO:
- [ ] enable teacher forcing randomization
- [ ] perform validation during the training procedure
- [ ] improve the code quality
- [ ] work in batches
- [x] check border conditions
- [ ] improve progress reporting
- [x] try tensorboard output
- [x] switch to DVC-based experimentation
- [ ] implement sequence padding to enable variable sequence length capability in the model

In [4]:
config = {}
with open("00-reversal-config.json", 'r') as ch:
    config = json.load(ch)

src_sequences = torch.randint(config['VOCAB_SIZE'] - 2, [config['SAMPLES'], config['SEQ_LENGTH']])
reversed_sequences = src_sequences.flip(1)
print(src_sequences[:10])
print(reversed_sequences[:10])

tensor([[4, 7, 5, 0, 3, 3, 3, 7, 1, 3],
        [5, 2, 4, 7, 6, 0, 0, 4, 2, 1],
        [6, 7, 7, 6, 0, 1, 5, 1, 5, 0],
        [1, 4, 3, 0, 3, 5, 6, 7, 7, 0],
        [2, 3, 0, 1, 3, 5, 3, 3, 6, 7],
        [0, 1, 1, 1, 7, 0, 7, 2, 4, 7],
        [3, 6, 3, 2, 7, 4, 2, 0, 0, 4],
        [5, 5, 6, 0, 4, 1, 7, 4, 1, 2],
        [2, 7, 0, 1, 1, 7, 1, 1, 3, 6],
        [7, 3, 6, 2, 3, 0, 6, 3, 5, 4]])
tensor([[3, 1, 7, 3, 3, 3, 0, 5, 7, 4],
        [1, 2, 4, 0, 0, 6, 7, 4, 2, 5],
        [0, 5, 1, 5, 1, 0, 6, 7, 7, 6],
        [0, 7, 7, 6, 5, 3, 0, 3, 4, 1],
        [7, 6, 3, 3, 5, 3, 1, 0, 3, 2],
        [7, 4, 2, 7, 0, 7, 1, 1, 1, 0],
        [4, 0, 0, 2, 4, 7, 2, 3, 6, 3],
        [2, 1, 4, 7, 1, 4, 0, 6, 5, 5],
        [6, 3, 1, 1, 7, 1, 1, 0, 7, 2],
        [4, 5, 3, 6, 0, 3, 2, 6, 3, 7]])


In [3]:
src_sequences_one_hot = torch.zeros((config['SAMPLES'], config['SEQ_LENGTH'], config['VOCAB_SIZE'])).scatter(2, src_sequences.unsqueeze(2), 1.0)

In [4]:
class ReverseEncoder(nn.Module):
    def __init__(self, seq_length, vocab_size, rec_layers_count):
        super(ReverseEncoder, self).__init__()
        self.seq_length = seq_length
        self.vocab_size = vocab_size
        self.rec_layers_count = rec_layers_count 
        self.rec_layers = nn.RNN(
            input_size = vocab_size, 
            hidden_size = seq_length**2,
            nonlinearity = "tanh",
            num_layers = rec_layers_count,
            batch_first = True
        )
        self.hidden_state = self.init_hidden_state()
        
    def init_hidden_state(self):
        return torch.randn((self.rec_layers_count, 1, self.seq_length**2))
        
    def forward(self, input_sequence):
        post_recurrent, hidden = self.rec_layers(input_sequence, self.hidden_state)

        return post_recurrent, hidden
    
class ReverseDecoder(nn.Module):
    def __init__(self, seq_length, vocab_size, rec_layers_count):
        super(ReverseDecoder, self).__init__()
        self.seq_length = seq_length
        self.vocab_size = vocab_size
        self.rec_layers_count = rec_layers_count 
        self.rec_layers = nn.RNN(
            input_size = vocab_size, 
            hidden_size = seq_length**2,
            nonlinearity = "tanh",
            num_layers = rec_layers_count,
            batch_first = True
        )
        self.output = nn.Linear(seq_length*seq_length, vocab_size)
        
    def forward(self, input_sequence, hidden_state):
        post_recurrent, hidden = self.rec_layers(input_sequence, hidden_state)
        item_probs = F.log_softmax(self.output(post_recurrent), dim=2)
        return item_probs, hidden

In [5]:
enc_model = ReverseEncoder(config['SEQ_LENGTH'], config['VOCAB_SIZE'], 2)
dec_model = ReverseDecoder(config['SEQ_LENGTH'], config['VOCAB_SIZE'], 2)

In [6]:
print(enc_model)
print(dec_model)

ReverseEncoder(
  (rec_layers): RNN(10, 100, num_layers=2, batch_first=True)
)
ReverseDecoder(
  (rec_layers): RNN(10, 100, num_layers=2, batch_first=True)
  (output): Linear(in_features=100, out_features=10, bias=True)
)


In [7]:
loss_function = nn.NLLLoss()
optimizer = torch.optim.Adam(
    list(enc_model.parameters()) + list(dec_model.parameters()), 
    lr = config['LEARNING_RATE']
)
writer = SummaryWriter()

In [8]:
SOS = torch.tensor(config['VOCAB_SIZE'] - 2)
EOS = torch.tensor(config['VOCAB_SIZE'] - 1)
SOS_filler = torch.cat((torch.zeros(config['VOCAB_SIZE'] - 2), torch.tensor([1.0, 0.0])))
EOS_filler = torch.cat((torch.zeros(config['VOCAB_SIZE'] - 1), torch.tensor([1.0])))

In [9]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [10]:
losses = {}
i = 0
for epoch in range(config['EPOCHS']):
    losses[epoch] = []
    with tqdm(list(chunks(list(zip(src_sequences_one_hot, reversed_sequences)), config['BATCH_SIZE']))) as cit:
        batch_element = 0
        for chunk in cit:
            loss = 0
            for (sequence, sequence_y) in chunk:
                X, y = sequence.unsqueeze(0), sequence_y
                single_batch_result_out, hidden = enc_model(X)
                Xss = torch.cat((SOS_filler.unsqueeze(0), X.squeeze(0)), dim=0)
                yss = torch.cat((y, EOS.unsqueeze(0)), dim=0).unsqueeze(1)
                for X_char, y_char in zip(Xss, yss):
                    y_pred, hidden = dec_model(X_char.unsqueeze(0).unsqueeze(0), hidden)
                    cur_loss = loss_function(y_pred.squeeze(0), y_char)
                    loss += cur_loss
                    i += 1
            enc_model.zero_grad()
            dec_model.zero_grad()
            loss.backward()
            losses[epoch].append(loss.tolist())
            writer.add_scalar("loss", loss.tolist(), global_step = epoch*config['BATCH_SIZE'] + batch_element)
            writer.add_scalar("epoch", epoch, global_step = epoch*config['BATCH_SIZE'] + batch_element)            
            optimizer.step()
            cit.set_postfix({
                'epoch': f"{epoch+1}/{config['EPOCHS']}", 
                'mean_loss': sum(losses[epoch])/len(losses[epoch]),
                'last_loss': losses[epoch][-1]
            })
            batch_element += 1


100%|██████████| 8/8 [00:04<00:00,  1.88it/s, epoch=15/15, mean_loss=1.06e+3, last_loss=886]    


In [11]:
def reverse_sequence(seq, model):
    seq_one_hot = torch.zeros((1, config['SEQ_LENGTH'], config['VOCAB_SIZE'])).scatter(2, seq.unsqueeze(2), 1.0)
    (_, hidden) = model[0](seq_one_hot)
    result = []
    out, hidden = model[1](SOS_filler.unsqueeze(0).unsqueeze(0), hidden)
    result.append(torch.argmax(out, dim=2).squeeze(0).squeeze(0).tolist())
    for seq_char in seq_one_hot.squeeze(0):
        out, hidden = model[1](seq_char.unsqueeze(0).unsqueeze(0), hidden)
        result.append(torch.argmax(out, dim=2).squeeze(0).squeeze(0).tolist())
    return result[:-1]

In [26]:
def const_generator(character_to_generate):
    while True:
        yield character_to_generate

def up_stairs_generator(cap = config['VOCAB_SIZE'] - 2, cur_character=0):
    while True:
        cur_character += 1
        if cur_character == cap + 1:
            cur_character = 0
        yield cur_character
        
def down_stairs_generator(cap = config['VOCAB_SIZE'] - 2, cur_character=0):
    while True:
        cur_character -= 1
        if cur_character < 0:
            cur_character = cap
        yield cur_character
        
def two_way_stairs_generator(cap = config['VOCAB_SIZE'] - 2, cur_character=0):
    delta = 1
    while True:
        cur_character += delta
        if cur_character >= cap:
            delta = -1
            cur_character = cap
        if cur_character <= 0:
            delta = 1
        yield cur_character



test_cases = []
for el in range(config['VOCAB_SIZE']):
    test_cases.append(list(itertools.islice(const_generator(el), config['SEQ_LENGTH'])))
    
for el in range(config['VOCAB_SIZE']):
    test_cases.append(list(itertools.islice(up_stairs_generator(el), config['SEQ_LENGTH'])))
    
for el in range(config['VOCAB_SIZE']):
    test_cases.append(list(itertools.islice(down_stairs_generator(el), config['SEQ_LENGTH'])))
        
for el in range(config['VOCAB_SIZE']):
    test_cases.append(list(itertools.islice(two_way_stairs_generator(el), config['SEQ_LENGTH'])))
    
for el in range(config['VOCAB_SIZE'] - 2, 0, -1):
    test_cases.append(list(itertools.islice(two_way_stairs_generator(el, cur_character = el), config['SEQ_LENGTH'])))


In [27]:
from difflib import SequenceMatcher

scores = ([SequenceMatcher(a = t[0].tolist(), b = reverse_sequence(t, (enc_model, dec_model))[::-1]).ratio() for t in test_cases])
score = sum(scores)/len(scores)

AttributeError: 'int' object has no attribute 'tolist'

In [ ]:
with open("00-seq-reversal-score.json", 'r') as ch:
    config = json.dump({"score": score}, ch)